In [1]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from Functions import *

In [2]:
# Importing credentials for Strava's API
from Credentials import StravaCredentials

# Copy and paste this link in your browser and extract code -> #https://www.strava.com/oauth/authorize?client_id=99205&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
data = StravaCredentials.data

# In case you need to get the code again uncomment these:
# webbrowser.open(f"https://www.strava.com/oauth/authorize?client_id={data['client_id']}&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all")
data['code'] = input("From the web broswer enter the code:")

In [2]:
# Setting up parameters for gspread - updating google sheet

service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'
myscope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']

mycred = ServiceAccountCredentials.from_json_keyfile_name(service_file_path,myscope) # type: ignore
client = gspread.authorize(mycred)
mysheet = client.open('workout-data').sheet1
list_of_row = mysheet.get_all_records()
all_workouts_df = pd.DataFrame(list_of_row)

In [22]:
df = all_workouts_df.copy()
df = df[['sport_type','pace']]
df.head(5)


print(all_workouts_df['sport_type'].unique())

['Functional-Cardio Workout' 'Run' 'Ride' 'Hike' 'TrailRun' 'Yoga' 'Swim'
 'Kayaking' 'MountainBikeRide' 'WeightTraining' 'Walk' 'Rowing'
 'AlpineSki']


In [18]:
pace_conditions = [
(df['sport_type'].isin(['Run'])) & (df['pace'] >= 5) & (df['pace'] < 6), #20
(df['sport_type'].isin(['Run'])) & (df['pace'] >= 6) & (df['pace'] < 7), #15
(df['sport_type'].isin(['Run'])) & (df['pace'] >= 7) & (df['pace'] < 10), #10
(df['sport_type'].isin(['Run'])) & (df['pace'] > 10), #5
(df['sport_type'].isin(['TrailRun'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['TrailRun'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['TrailRun'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['TrailRun'])) & (df['pace'] > ),
(df['sport_type'].isin(['Hike'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Hike'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Hike'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Hike'])) & (df['pace'] > ),
(df['sport_type'].isin(['Swim'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Swim'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Swim'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Swim'])) & (df['pace'] > ),
(df['sport_type'].isin(['MountainBikeRide'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['MountainBikeRide'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['MountainBikeRide'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['MountainBikeRide'])) & (df['pace'] > ),
(df['sport_type'].isin(['Ride'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Ride'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Ride'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['Ride'])) & (df['pace'] > ),
(df['sport_type'].isin(['AlpineSki'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['AlpineSki'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['AlpineSki'])) & (df['pace'] >= ) & (df['pace'] < ),
(df['sport_type'].isin(['AlpineSki'])) & (df['pace'] > )]

pace_conditions_values = [20, 15, 10, 5]

df['pace_score'] = np.select(pace_conditions, pace_conditions_values)

In [19]:
df.loc[df['sport_type'] == 'Run'].head(5)

,sport_type,pace,pace_score
2,Run,6.35,15
4,Run,5.52,20
10,Run,5.36,20
12,Run,5.52,20
21,Run,5.38,20


In [ ]:
    distance_conditions = [
        (df['distance']==0), # 1
        (df['sport_type'].isin(['Run', 'TrailRun'])) & (df['distance'] >= 0) & (df['distance'] < 5), # 5
        (df['sport_type'].isin(['Run', 'TrailRun'])) & (df['distance'] >= 5) & (df['distance'] < 10), # 10 
        (df['sport_type'].isin(['Run', 'TrailRun'])) & (df['distance'] >= 10) & (df['distance'] < 13), # 25 
        (df['sport_type'].isin(['Run', 'TrailRun'])) & (df['distance'] >= 13), # 30
        (df['sport_type'].isin(['Ride', 'MountainBikeRide'])) & (df['distance'] >= 1) & (df['distance'] < 5), # 5
        (df['sport_type'].isin(['Ride', 'MountainBikeRide'])) & (df['distance'] >= 5) & (df['distance'] < 8.5), # 10
        (df['sport_type'].isin(['Ride', 'MountainBikeRide'])) & (df['distance'] >= 8.5) & (df['distance'] < 12), # 15
        (df['sport_type'].isin(['Ride', 'MountainBikeRide'])) & (df['distance'] >= 12) & (df['distance'] < 15), # 20
        (df['sport_type'].isin(['Ride', 'MountainBikeRide'])) & (df['distance'] >= 15), # 25
        (df['sport_type'].isin(['AlpineSki']) & (df['distance'] >= 10) & (df['distance'] < 15)), # 10
        (df['sport_type'].isin(['AlpineSki']) & (df['distance'] >= 15) & (df['distance'] < 20)), # 15
        (df['sport_type'].isin(['AlpineSki']) & (df['distance'] >= 20)), # 20
        (df['sport_type'].isin(['Swim'])) & (df['distance'] >= 0.10) & (df['distance'] < 0.20), # 10
        (df['sport_type'].isin(['Swim'])) & (df['distance'] >= 0.20) & (df['distance'] < 0.30), # 15
        (df['sport_type'].isin(['Swim'])) & (df['distance'] >= 0.30) & (df['distance'] < 0.35), # 20
        (df['sport_type'].isin(['Swim'])) & (df['distance'] >= 0.35) & (df['distance'] < 0.40), # 25
        (df['sport_type'].isin(['Swim'])) & (df['distance'] >= 0.40), # 30
        (df['distance']> 0.5) & (df['distance'] < 1), # 1
        (df['distance']> 1) & (df['distance'] < 2), # 2
        (df['distance']> 2) & (df['distance'] < 3), # 3
        (df['distance']> 3) & (df['distance'] < 4), # 4
        (df['distance']> 4) # 5
        ]

    distance_conditions_values = [1, 5, 10, 25, 30, # running
                                5, 10, 15, 20, 25, # biking
                                10, 15, 20, # skiing
                                10, 15, 20, 25, 30, # swimming
                                1, 2, 3, 4, 5 # special activitie 
                                ]

    # applying conditions and values
    df['distance_score'] = np.select(distance_conditions, distance_conditions_values)